### This notebook was an attempt to crawl for company responses to the tweets, but after hours, no output was obtained at all, and the code was still running. Therefore the attempt was recorded yet, albeit in vain.

Twitter's API *doesn't allow you to get replies to a particular tweet*. Strange but true. But you can use Twitter's Search API to search for tweets that are directed at a particular user, and then search through the results to see if  any are replies to a given tweet. The big caveat 
here is that the **search API only returns results for the last 7 days**.
<br/> 
<br/>
reference: https://gist.github.com/edsu/54e6f7d63df3866a87a15aed17b51eaf

In [0]:
import os
import tweepy
import pandas as pd
import numpy as np

In [3]:
import getpass # prevents echo that is unhidden

# allow users to key in tokens here:
consumerkey = getpass.getpass("Consumer Key: ")
consumersecret = getpass.getpass("Consumer Secret: ")
accesstoken = getpass.getpass("Access Token: ")
accesstokensecret = getpass.getpass("Access Token Secret: ")

Consumer Key: ··········
Consumer Secret: ··········
Access Token: ··········
Access Token Secret: ··········


In [0]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumerkey, consumersecret)
auth.set_access_token(accesstoken,accesstokensecret)

api = tweepy.API(auth, wait_on_rate_limit=True) # twitter api has a rate limit, and adding this helps the code to obey this limit 
                                                # instead of throwing error

# without a cursor, to properly handle the Tweepy SearchResults object, i need this:
# api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True) 
# because the object returned to me is a rich python object, and doing this allows me to parse it as json

## Company responses
We need to indicate in our search api that we are looking for "to: " customer and "from: " company
<br/>
*reference: https://developer.twitter.com/en/docs/tweets/search/overview/premium#AvailableOperators*

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
# importing the customer data

cust_tweet_path = "/content/drive/My Drive/BT4222/Data/cust_tweets.csv"

cust_tweets = pd.read_csv(cust_tweet_path)
cust_tweets.head()

,Unnamed: 0,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id,customers_of
0,0,1249193154512916480,375541562,False,2020-04-12 04:30:31,@Jetstar_Asia Please check your DM. Thanks @Je...,1.249190e+18,JetStar Asia
1,1,1249189062759665667,375541562,False,2020-04-12 04:14:16,@Jetstar_Asia Pleas help me. I have also tried...,1.249186e+18,JetStar Asia
2,2,1249040470401060864,1024338385404682246,True,2020-04-11 18:23:48,RT @monilsj: Hi @Jetstar_Asia! I have noticed ...,NaN,JetStar Asia
3,3,1249038696214458369,286610770,True,2020-04-11 18:16:45,Hi @Jetstar_Asia! I have noticed that Air Asia...,NaN,JetStar Asia
4,4,1248566696798867456,1048797604019036160,True,2020-04-10 11:01:12,RT @RayMSLim: @Jetstar_Asia why do you make it...,NaN,JetStar Asia


In [7]:
cust_tweets = cust_tweets.drop(columns=['Unnamed: 0'])
cust_tweets.shape

(7643, 7)

In [0]:
# purpose: to use when looping through all customers to get company response
# goal: to get the company's response to this user tweet

def getCompanyResponse(author_id, company_handle):
  tweet_id = []
  authod_id = []
  inbound = [] # inbound are tweets that arent in reply to anything
  created_at = []
  text = []
  # reponses_tweet_id = []
  in_response_to_tweet_id = []

  # convert author_id to screen_name
  screen_name = api.get_user(author_id).screen_name

  for t in tweepy.Cursor(api.search, q="to:"+screen_name+" AND from:"+company_handle, count=500, lang='en').items():
    tweet_id.append(t.id_str)
    authod_id.append(t.user.id_str)
    inbound.append(t.in_reply_to_user_id_str == None)
    created_at.append(t.created_at)
    text.append(t.text)
    # responses_tweet_id.append(t.reply_count)
    in_response_to_tweet_id.append(t.in_reply_to_status_id_str)
  
  output = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

  return output

## AirAsia Support

In [0]:
handle = "AirAsiaSupport"

In [11]:
aa_cust = cust_tweets.loc[cust_tweets["customers_of"] == "AirAsia"]
aa_cust.shape

(6281, 7)

In [0]:
aa_response = pd.DataFrame(data={'tweet_id': [], 'authod_id': [], 'inbound': [], 'created_at': [], 'text': [], 'in_response_to_tweet_id': []})

for cust in aa_cust["authod_id"]:
  result = getCompanyResponse(cust, handle)
  aa_response = pd.concat([aa_response, result])

aa_response.shape